In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import unicodedata
import random
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU')

Training on GPU!


In [ ]:
# 몇 개의 단어를 context vec으로 사용할지 설정

context_num = 2

In [ ]:
lines = open("/content/textbook", "r").read().split('\n')
# lines = lines[:1000]

In [ ]:
test_sentence = [word_tokenize(s) for s in lines]
print(test_sentence[0])

['sara', ':', 'my', 'name', 'is', 'sara', '.']


In [ ]:
vocab = []
for sent in test_sentence:
    for word in sent: vocab.append(word)
        
vocab = set(vocab)
voc_len=len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(voc_len)

1552


In [ ]:
MAX_LEN = 75

In [ ]:
input_ids = []
for sent in test_sentence:
    input_ids.append([word_to_ix[w] for w in sent])

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[1005  657  652  486  697 1209  980 1520 1187    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
inp = []
tar = []

for sentence in input_ids:
    for i in range(len(sentence)-context_num):
        context = sentence[i:i+context_num]
        target = sentence[i+context_num]
        if sum(context.tolist())+target==0: continue
        inp.append(context)
        tar.append(torch.tensor([target]))

In [ ]:
inp = torch.tensor(inp)
data_len = len(inp)
print(data_len)

2205126


In [ ]:
for i, t in zip(inp, tar):
    print(i.size())
    print(i)
    print(t)
    break

torch.Size([4])
tensor([1005,  657,  652,  486])
tensor([697])


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size*context_num, hidden_size, n_layers,batch_first=True,
                          bidirectional=False)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hc):
        input = self.encoder(input.view(1, -1))
        output, (hidden, context) = self.lstm(input.view(1, 1, -1), hc)        
        output = self.decoder(output.view(1, -1))        
        return output, hidden, context

    def init_hidden(self):
        return Variable((torch.zeros(self.n_layers, 1, self.hidden_size)))
    
    def init_context(self):
        return Variable((torch.zeros(self.n_layers, 1, self.hidden_size)))
    

In [ ]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
def train(inp, target):
    decoder.zero_grad()
    hidden = decoder.init_hidden().cuda()
    context = decoder.init_context().cuda()
    loss = 0
    output, hidden, context = decoder(inp.cuda(), (hidden, context))
    
    loss += criterion(output, target.cuda())
    loss.backward()
    decoder_optimizer.step()
    
    return loss.data.item()

In [ ]:
def generate(prime_str='the name of the dog', predict_len=75, temperature=0.8):
    torch.no_grad()

    hidden = decoder.init_hidden().cuda()
    context = decoder.init_context().cuda()
    hc = (hidden, context)

    for p in range(predict_len):
        prime_input = torch.tensor([word_to_ix[w] for w in prime_str.split()], dtype=torch.long).cuda()
        inp = prime_input[-1*context_num:]

        if torch.sum(inp) == 0: break  # 종료 조건
            
        output, hidden, context = decoder(inp, hc)
        
        # Sample from the network as a multinomial distribution
        output_dist = F.softmax(output.data).view(-1).div(temperature).exp()
        
        top_i = torch.multinomial(output_dist, 1)[0]
    
        
        # Add predicted word to string and use as next input
        predicted_word = list(word_to_ix.keys())[list(word_to_ix.values()).index(top_i)]
        prime_str += " " + predicted_word

    return prime_str

In [ ]:
n_epochs = 10
print_every = 2
plot_every = 2
hidden_size = 100
n_layers = 2
lr = 0.015

decoder = LSTM(voc_len, hidden_size, voc_len, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()
decoder.train()

start = time.time()
all_losses = []
decoder.cuda()


for epoch in range(1, n_epochs + 1):
    print("Epoch: ", epoch)
    loss_avg = 0
    for i,t in zip(inp, tar):
        loss = train(i,t)       
        loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%)]' % (time_since(start), epoch, epoch / n_epochs * 100))
        print(generate())

    if epoch % plot_every == 0:
        all_losses.append((loss_avg/data_len)/ plot_every)
        loss_avg = 0

In [ ]:
all_losses

In [ ]:
torch.save(decoder, 'lstmmodel.pt')

## Generate sentence

In [ ]:
print(generate('the name of the dog', 75, temperature=3))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


the name of the dog by dogs shooting ! last computer tteok know comes flowers rocket great reading sound visit birds third don'y let waht angry than juho 22nd teach fred made isstv they floor rode 205 everything mon mine jack sure enjoy $ letters snowing ketchup beethoven painting roe hit bees last watch favorite echo loves lost number three tired sons t-shirts song sit hercules whoose dog anu sweet thursday librarian borte daddy watched will mrs.lee magic swimming quizmaster club l ? viking noodles tuesday . thousand rockets ... every salty 11:00 dad rock history watching bongsu gyeongsang-do house ideas studying annika takes stopped
